In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cuvis
from cuvis_ai.pipeline import Graph
from cuvis_ai.distance import SpectralAngle
from cuvis_ai.data import PublicDataSets
from IPython.display import clear_output
%matplotlib inline

# Public Data Loading and SAM tracking

in this example a public dataset is loaded containing measurements of an Aquarium obtained with a cubert hyperspectral camera. After loading the data, the first image is displayed and a reference spectrum for a spectral angle mapper node is obtained. Using a SAM trained on the reference spectrum, the images of the video are classified and plotted.

## Download public dataset

In [ ]:
data_path = "../../test_data/Aquarium"
data = PublicDataSets.download_dataset("Aquarium", download_path=data_path)

## Create graph and process measurement
The data in the Aquarium dataset is not processed yet. This reduces the size considerably.

In [ ]:
graph = Graph('Aquarium_dist')
session = cuvis.SessionFile(data_path + '/Auto_004.cu3s')
mesu = session.get_measurement(1)
pc = cuvis.ProcessingContext(session)
pc.apply(mesu)
cube = mesu.data.get("cube")

## Variable assignment

In [ ]:
cube_array = cube.array
wave_lengths = cube.wavelength
spectrum_y_coord = 28
spectrum_x_coord = 35

## Getting a reference spectrum from the measurement for the SAM

In [ ]:
ref_spectrum = np.array([cube_array[spectrum_y_coord, spectrum_x_coord, :]])
plt.figure()
plt.plot(wave_lengths, ref_spectrum.T)
plt.show()

## Display the position at which the spectrum was extracted 

In [ ]:
plt.figure()
plt.imshow(cube_array[:, :, 31], cmap='binary')
plt.scatter(spectrum_x_coord, spectrum_y_coord, s=5, c='r')
plt.show()

## Create the SAM and set a maximal allowed distance from the reference spectrum 

In [ ]:
SAM = SpectralAngle(ref_spectrum)
max_distance = 0.2

## Iterate through the session file and classify all measurements

Here a custom colormap is used to only show pixels which are classified and make the rest of the mask transparent. The SAM expects normalized images, so we need to normalize our measurements before classifying them.

In [ ]:
cmap = plt.get_cmap('Reds')
cmap.set_under('k', alpha=0)
for i in range(10,100,2):
    clear_output()
    mesu = session.get_measurement(i)
    pc.apply(mesu)
    cube = mesu.data.get("cube").array
    cube_norm = np.linalg.norm(cube)
    cube_norm = cube/cube_norm
    out = SAM.forward(cube_norm)
    out[out > max_distance] = 0
    fig = plt.figure()
    ax = fig.add_subplot(111)
    img = cube[:,:,31] 
    ax.imshow(img,cmap="binary")
    ax.imshow(out, cmap, alpha=1, vmin=out.max()*max_distance)
    plt.show()
    plt.pause(0.1)   